In [1]:
# importando as principais bibliotecas
from langchain_openai import OpenAI
from dotenv import load_dotenv

In [2]:
# carregando as variáveis que estão no .env
load_dotenv()

True

### Memory

É a capacidade do modelo de guardar as informações e se lembrar de conversas passadas.

In [14]:
from langchain.memory import ConversationBufferMemory

In [15]:
# primeiro vamos instanciar e iniciar a memória
memory = ConversationBufferMemory()

In [9]:
# agora adicionando algumas conversas a esta memoria
# veja que é deivdido entre falas do usuário e falas do modelo
memory.chat_memory.add_user_message("Olá!")
memory.chat_memory.add_ai_message("Oi, como vai?")


In [10]:
# vamos o q já temos dentro da memória
memory.load_memory_variables({})

{'history': 'Human: Olá!\nAI: Oi, como vai?'}

**Criando uma conversa com memória**

Vamos agora unir o chat à memória através de uma chain

In [16]:
# importando as libs necessárias
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.conversation.base import ConversationChain

In [17]:
# criando o chat
chat = ChatOpenAI()

In [18]:
# criando a memória
memory = ConversationBufferMemory()

In [19]:
# e agora juntando os dois numa única chain
# fazemos isso instanciando o objeto ConversationChain()
conversation = ConversationChain(
    llm=chat,
    memory=memory,
    # deixando o verbose como True retorna toda a interação
    # com o modelo. Isso facilita para debugar.
    # quando está false, ele retorna somente a resposta do modelo
    verbose=True,
    
)

In [20]:
# veja como fica a saída com o verbose = True
# ele mostra o prompt montado pelo chain para otimizar a memória do modelo
conversation.predict(input="opa. Meu nome é Kaio, \
                     e quero q vc seja meu conselheiro de carreira.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: opa. Meu nome é Kaio,                      e quero q vc seja meu conselheiro de carreira.
AI:

> Finished chain.


'Olá Kaio! Fico feliz em ser seu conselheiro de carreira. Estou aqui para ajudá-lo no que precisar. Como posso ajudá-lo hoje?'

In [21]:
# veja que o chain vai guardando o histórico de conversa
conversation.predict(input="Que bom, quero falar sobre analise de dados.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: opa. Meu nome é Kaio,                      e quero q vc seja meu conselheiro de carreira.
AI: Olá Kaio! Fico feliz em ser seu conselheiro de carreira. Estou aqui para ajudá-lo no que precisar. Como posso ajudá-lo hoje?
Human: Que bom, quero falar sobre analise de dados.
AI:

> Finished chain.


'Claro, Kaio! A análise de dados é um campo muito interessante. Existem várias ferramentas e técnicas que podem ser utilizadas para analisar dados, como estatísticas descritivas, regressão, análise de séries temporais, machine learning, entre outras. Você tem alguma pergunta específica sobre análise de dados em mente? Eu posso te ajudar a entender melhor os conceitos ou aconselhá-lo sobre como se aprofundar nesse campo.'

In [22]:
# podemos verificar o histórico acessando as variáveis da memória
memory.load_memory_variables({})

{'history': 'Human: opa. Meu nome é Kaio,                      e quero q vc seja meu conselheiro de carreira.\nAI: Olá Kaio! Fico feliz em ser seu conselheiro de carreira. Estou aqui para ajudá-lo no que precisar. Como posso ajudá-lo hoje?\nHuman: Que bom, quero falar sobre analise de dados.\nAI: Claro, Kaio! A análise de dados é um campo muito interessante. Existem várias ferramentas e técnicas que podem ser utilizadas para analisar dados, como estatísticas descritivas, regressão, análise de séries temporais, machine learning, entre outras. Você tem alguma pergunta específica sobre análise de dados em mente? Eu posso te ajudar a entender melhor os conceitos ou aconselhá-lo sobre como se aprofundar nesse campo.'}

**ConversationBufferWindowMemory**

É um tipo de memória diferente da anterior. Cada aplicação pode pedir um tipo diferente de memporia.

Esta memória só guarda um numero específico de conversas passadas. Essa quantidade de conversas a serem guardadas é indicada como argumento no momento em q estamos instanciando a memória.

In [23]:
from langchain.memory import ConversationBufferWindowMemory

In [25]:
# vamos recirar alguns objetos para ilustrar como funciona essa memória

# instanciando a memoria que acabamos de importar
# veja que aqui queremos q lembre somente de uma conversa anterior
memory = ConversationBufferWindowMemory(k=1)

# novamente juntando o modelo (chat) com o novo tipo de memoria criado
conversation = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True,
    
)

In [26]:
# vamos exemplificar aqui como funciona o k
# eu começo falando meu nome para o modelo
conversation.predict(input="Olá meu nome é Kaio.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Olá meu nome é Kaio.
AI:

> Finished chain.


'Olá Kaio! É um prazer conhecê-lo. Como posso ajudá-lo hoje?'

In [27]:
# o modelo responde mencionando meu nome
conversation.predict(input="Como vai?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Olá meu nome é Kaio.
AI: Olá Kaio! É um prazer conhecê-lo. Como posso ajudá-lo hoje?
Human: Como vai?
AI:

> Finished chain.


'Estou bem, obrigado por perguntar! Estou aqui para ajudá-lo com qualquer dúvida ou informação de que precisar. Como posso ajudá-lo hoje?'

In [28]:
# ao prosseguir o diálogo pergunto se ele sabe meu nome
# o modelo não sabe respondê-la, pois já se esqueceu q eu tinha falado na janela anterior
# veja no retorno abaixo como o prompt vem formatado
# a memoria só lembra de uma conversa anterior e o diálogo já se inicia no 'como vai?'
conversation.predict(input="Qual é o meu nome?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Como vai?
AI: Estou bem, obrigado por perguntar! Estou aqui para ajudá-lo com qualquer dúvida ou informação de que precisar. Como posso ajudá-lo hoje?
Human: Qual é o meu nome?
AI:

> Finished chain.


'Desculpe, mas eu não tenho essa informação. Como posso ajudá-lo com algo mais?'

**ConversationTokenBufferMemory**

Aqui a memória é limitada pelo tamanho das conversas, ou seja, toda a conversa anterior não deve passar de um número X de tokens. Essa quantidade é indicada como argumento no momento em ue instanciamos a memmoria

In [29]:
from langchain.memory import ConversationTokenBufferMemory

In [31]:
# vamos fazer aqui o mesmo processo das anteriores
# e recirar alguns objetos para ilustrar como funciona essa memória

# instanciando a memoria que acabamos de importar
# ela pede 2 argumentos principais, o modelo
# e o máximo de tokens que a memória deverá se lembrar
memory = ConversationTokenBufferMemory(llm=chat, max_token_limit=250)

# novamente juntando o modelo (chat) com o novo tipo de memoria criado
conversation = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True,
    
)

**ConversationSummaryBufferMemory**

Este tipo de memória aqui faz um resumo de toda conversa anterior. Ele inclui esse resumo no prompt e já passa o contexto para o modelo. As anteriores mostravam todo o diálogo enquanto essa aqui traz apenas o resumo do q foi conversado até então. A quantidade máxima de tokens é para indicar o tamanho que o resumo deverá ter.

In [44]:
from langchain.memory import ConversationSummaryBufferMemory

In [45]:
# vamos fazer aqui o mesmo processo das anteriores
# e recirar alguns objetos para ilustrar como funciona essa memória

# instanciando a memoria que acabamos de importar
# assim como a anterior, ela tbm pede um modelo como argumento
memory = ConversationSummaryBufferMemory(llm=chat, max_token_limit=250)

# novamente juntando o modelo (chat) com o novo tipo de memoria criado
conversation = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True,
    
)

In [46]:
# veja que no inicio ele lembra do diálogo como nas outras memórias
conversation.predict(input="Opa, meu nome é Kaio")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Opa, meu nome é Kaio
AI:

> Finished chain.


'Olá, Kaio! Eu sou um assistente de inteligência artificial. Como posso ajudar você hoje?'

In [47]:
# mas assim que a conversa cresce ele vai precisar fazer um resumo
# aqui eu pedi pra ele contar uma historia grande, pra ultrapassar os tokens
conversation.predict(input="me conte uma historia de 500 palavras")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Opa, meu nome é Kaio
AI: Olá, Kaio! Eu sou um assistente de inteligência artificial. Como posso ajudar você hoje?
Human: me conte uma historia de 500 palavras
AI:

> Finished chain.


'Claro, Kaio! Vou te contar uma história sobre um jovem aventureiro chamado Miguel. Miguel era um garoto curioso e destemido que sempre sonhou em explorar o mundo. Um dia, ele decidiu sair de sua pequena cidade e embarcar em uma jornada épica.\n\nMiguel começou sua jornada nas montanhas geladas do norte, onde enfrentou tempestades de neve e criaturas selvagens. Ele conheceu tribos antigas e aprendeu seus segredos de sobrevivência. Depois de semanas de viagem, Miguel finalmente chegou à costa do oceano, onde viu pela primeira vez as ondas gigantes e os navios que cruzavam os mares.\n\nDecidido a se tornar um grande navegador, Miguel se juntou a uma tripulação de piratas e navegou pelos sete mares em busca de tesouros perdidos. Ele enfrentou batalhas navais, navegou por mares traiçoeiros e explorou ilhas desconhecidas. Ao longo de sua jornada, Miguel fez amigos leais e inimigos perigosos.\n\nEm uma de suas aventuras, Miguel descobriu um mapa antigo que levava a uma ilha misteriosa. Ele e

In [48]:
# veja q agora na próxima interação o prompt feito automaticamente
# já traz um resumo, em vez do histórico do dialogo
conversation.predict(input="Qual seu filme preferido?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human introduces himself as Kaio to the AI, who responds with a greeting and offers assistance. Kaio asks the AI to tell a 500-word story, and the AI tells a tale about a young adventurer named Miguel. Miguel embarks on a journey through snowy mountains, encounters ancient tribes, joins pirates to explore the seas, and discovers a legendary treasure on a mysterious island. After years of adventures, Miguel returns home as a hero, inspiring future generations with his courage and determination.
Human: Qual seu filme preferido?
AI:

> Finished chain.


'Eu não assisto filmes, então não tenho um filme preferido. Mas posso te recomendar alguns baseados em informações que tenho acessado. Gosta de filmes de ação, comédia, romance, ficção científica ou algum outro gênero específico? Eu posso te ajudar a encontrar um filme que combine com seus interesses.'

### Chains - Encadeamento de prompts com Langchain

Chains são componentes essenciais, pois ela liga diversos módulos (prompts, llms, output parsers) de forma a construir uma aplicação robusta e complexa.

Anteriormente trabalhamos de forma independente a aprender as diversas formas de criar um prompt, instanciar um modelo, criar output parsers etc, agora vamos ligar todos eles numa chain.

Passar várias instruções para um unico prompt vai fazer com que o modelo performe mal. Por exemplo, pedir pra ele resumir um texto, depois pedir pra ele extrair alguma informação, depois pedir pra ele traduzir pra outro idioma...tudo isso funciona melhor se quebrarmos essas etapas em diversos prompts especializados e depois juntar todos em um chain. Porém existem vários tipo de chain diferentes, vamos explicar um pouco delas abaixo:

**Conversation Chain**

In [10]:
from langchain.chains.conversation.base import ConversationChain

In [29]:
# vamos construir uma chain como fizemos no exemplo da aula anterior

# vamos instanciar e criar um chat
chat = ChatOpenAI(model='gpt-3.5-turbo-0125')

# criando a memoria
memory = ConversationBufferMemory()

In [51]:
# agora juntando tudo numa chain simples
# basta fornecer os componentes criados como argumentos da chain
chain = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True
)

In [52]:
# agora veja a chain funcionando
# perceba tbm q ela cria um prompt para dar contexto ao modelo
chain.predict(input="Opa, testando..")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Opa, testando..
AI:

> Finished chain.


'Olá! Estou aqui para ajudar. O que você gostaria de saber ou discutir hoje?'

In [5]:
# agora vamos dizer q queremos fornecer um prompt para a chain
# vamos fazer isso agora
from langchain.prompts import PromptTemplate

In [55]:
# criando um prompt template
# a variável history deve ser inserida, pois ela representa o historico do chat
prompt_template = PromptTemplate.from_template(
'''
Essa é uma conversa amigável entre um humano e uma IA.
                                               
Conversa atual:

{history}

Human: {input}

AI:                                               
''')

In [57]:
# agora basta criar a chain como criada anteriormente
# a diferença é q agora vc vai forcener mais um argumento, o prompt
chain = ConversationChain(
    prompt=prompt_template,
    llm=chat,
    memory=memory,
    verbose=True
)

In [58]:
chain.predict(input="opa, testando!")



> Entering new ConversationChain chain...
Prompt after formatting:

Essa é uma conversa amigável entre um humano e uma IA.
                                               
Conversa atual:

Human: Opa, testando..
AI: Olá! Estou aqui para ajudar. O que você gostaria de saber ou discutir hoje?

Human: opa, testando!

AI:                                               


> Finished chain.


'Pode contar comigo para ajudar no que precisar. Estou aqui para conversar e interagir com você. Como posso ajudar hoje?'

In [60]:
# veja que podemos usar o método run em vez do predict
# ele só pede o comando direto, não precisa escrever o 'input'
chain.run("oi")

/home/kaio-peixoto/GitHub/asimov/aplicacoes-langchain/venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:

Essa é uma conversa amigável entre um humano e uma IA.
                                               
Conversa atual:

Human: Opa, testando..
AI: Olá! Estou aqui para ajudar. O que você gostaria de saber ou discutir hoje?
Human: opa, testando!
AI: Pode contar comigo para ajudar no que precisar. Estou aqui para conversar e interagir com você. Como posso ajudar hoje?

Human: oi

AI:                                               


> Finished chain.


'Olá! Como posso ajudar você hoje?'

**LLMChain**

In [8]:
from langchain.chains.llm import LLMChain

In [72]:
# vamos criar aqui o prompt para usar nesse chain
prompt_llm_chain = PromptTemplate.from_template(
"""Qual o melhor nome de empresa para uma empresa que desenvolve o produto: {produto}\
Retorne apenas um nome"""
)

In [70]:
# agora basta instanciar LLMChain, fornecer os argumentos e criar a chain
chain = LLMChain(llm=chat, memory=memory, prompt=prompt_llm_chain)

In [73]:
# veja que só precisamos fornecer a variável (produto)
produto = 'filtros de barro artesanais'
chain.run(produto)

'ArteBarro'

**SimpleSequentialChain**

Aqui tornamos o chain mais complexo do q o LLMChain. Vamos estruturar e combinar uma sequencia de instruções diferentes. O output de uma chain é passado para outra chain, de forma cascateada, de modo que quebramos a tarefas em vários passos para otimizar o resultado final.

In [143]:
from langchain.chains import SimpleSequentialChain

In [144]:
# vamos criar nosso primeiro prompt dessa sequencia
prompt_produto = PromptTemplate.from_template(
"""Qual o melhor nome de empresa para uma empresa que desenvolve o produto: {produto}\
Retorne apenas um nome"""
)

chain_produto = LLMChain(llm=chat, memory=memory, prompt=prompt_produto)

In [145]:
# segundo prompt, q vai utilizar o output da chain anterior
prompt_descricao_empresa = PromptTemplate.from_template(
"""Dado a empresa com o seguinte nome: {nome}. Dê\
uma descrição de até 50 palavras das atividades dessa empresa."""
)

chain_descricao_empresa = LLMChain(llm=chat, memory=memory, prompt=prompt_descricao_empresa)

In [146]:
# criando a chain sequencial
# veja que a sequencia como os chains são colocados na lista impacta no resultado
# esta sequential chain vai retornar o resultado da ultima chain da lista de chains
sequential_chain = SimpleSequentialChain(
    chains=[chain_produto, chain_descricao_empresa],
    verbose=True
)

In [147]:
# testando nossa chain
produto = 'filtros de barro artesanais'
sequential_chain.run(produto)



> Entering new SimpleSequentialChain chain...
ArteClayFiltros
A ArteClayFiltros é uma empresa especializada na fabricação e venda de filtros de argila para purificação de água. Com produtos de alta qualidade e eficiência, a empresa atua no mercado oferecendo soluções sustentáveis para o tratamento de água potável em residências e estabelecimentos comerciais.

> Finished chain.


'A ArteClayFiltros é uma empresa especializada na fabricação e venda de filtros de argila para purificação de água. Com produtos de alta qualidade e eficiência, a empresa atua no mercado oferecendo soluções sustentáveis para o tratamento de água potável em residências e estabelecimentos comerciais.'

**SequentialChain**

A diferença para o exemplo anterior é que essa chain não precisa estar na ordem para q uma consuma o output da outra. Em vez disso, nós vamos fornecer output_keys, q serão as variáveis produzidas pela chain. Os nomes das variáveis podem ser utilizadas em outras chains, eliminando a necessidade de estarem em ordem, como na SimpleSequentialChain.

In [21]:
from langchain.chains import SequentialChain

In [22]:
# vamos aqui utilizar os mesmos prompts do exemplo anterior
# veja que agora tem uma diferença, a presença do argumento output_key
# a outra diferença é q não usamos mais o argumento memory
# pois ele pode causar conflitos, como inconsistencia de variáveis,
# no fluxo de dados gerenciado pelo SequentialChain


prompt_produto = PromptTemplate.from_template(
"""Qual o melhor nome de empresa para uma empresa que desenvolve o produto:{produto}
Retorne apenas um nome""")

chain_produto = LLMChain(llm=chat, prompt=prompt_produto, output_key='nome')


prompt_descricao_empresa = PromptTemplate.from_template(
"""Dado a empresa com o seguinte nome: {nome} e que\
produz o seguinte produto: {produto}, dê uma descrição de até\
50 palavras das atividades dessa empresa."""
)

chain_descricao_empresa = LLMChain(llm=chat, prompt=prompt_descricao_empresa, 
                                   output_key='descricao_empresa')

# adicionando mais um prompt, o de tradução
prompt_traducao = PromptTemplate.from_template(
"""Crie um nome em russo para a empresa de nome {nome}
    que possui a seguinte descrição {descricao_empresa}""")

chain_traducao = LLMChain(llm=chat, prompt=prompt_traducao, output_key='traducao')

In [23]:
# agora vamos criar nossa SequentialChain
# note q agora temos alguns argumentos q não fizemos em outras chains
# precisamos declarar quem são as input variables e as output keys

chain = SequentialChain(
    chains=[chain_produto, chain_descricao_empresa, chain_traducao],
    input_variables=['produto'],
    output_variables=['nome', 'descricao_empresa', 'traducao'],
    verbose=True)

In [25]:
resposta = chain.invoke({'produto':'barbeador bruto da roça'})
resposta



> Entering new SequentialChain chain...

> Finished chain.


{'produto': 'barbeador bruto da roça',
 'nome': 'Barba Rústica',
 'descricao_empresa': 'A empresa Barba Rústica se destaca pela produção de barbeadores brutos da roça, feitos com materiais naturais e de alta qualidade. Com foco na praticidade e eficiência, a empresa oferece produtos que atendem às necessidades dos homens que buscam um estilo de vida mais autêntico e rústico.',
 'traducao': 'Брутальная Борода Грубая (Brutalnaya Boroda Grubaya)'}

In [27]:
# veja q o retorno é um dicionário
# portanto podemos capturar os resultados separadamente
resposta['produto']

'barbeador bruto da roça'

In [28]:
resposta['traducao']

'Брутальная Борода Грубая (Brutalnaya Boroda Grubaya)'

### RouterChains - Cadeias de Roteamento

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.llm import LLMChain

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

1 - Primeiro passo, criar uma lista de dicionarios dos prompts. Esses dicionários vão trazer informações como nome do prompt, descrição do q ele faz e o respectivo prompt_template.

2 - criando as chains: vamos usar os templates e criar as chains específicas de cada prompt. Essas chains vão todas para um dicionário de chains.

3 - criando os destinos. É aqui q o router vai se guiar para saber para qual chain direcionar cada input do usuário.



In [2]:
# temos aqui 3 prompts especializados
# cada um em uma matéria específica

fis_template = ChatPromptTemplate.from_template("""
Vc é um exímio professor de física.
Vc é ótimo em responder perguntas sobre física de forma concisa e fácil de entender.\
Quando não sabe a resposta para uma perginta, vc admite que não sabe.
                                                
Aqui está a pergunta: {input}""")

mat_template = ChatPromptTemplate.from_template("""
Vc é um exímio professor de matemática, a verdadeira reencarnação de Pitágoras.\
Vc é ótimo em responder perguntas sobre matemática de forma concisa e fácil de entender.\
Quando não sabe a resposta para uma perginta, vc admite que não sabe.
                                                
Aqui está a pergunta: {input}""")

hist_template = ChatPromptTemplate.from_template("""
Vc é um exímio professor de história, a verdadeira reencarnação de Eric Hobsbawm.\
Vc é ótimo em responder perguntas sobre história de forma concisa e fácil de entender.\
Quando não sabe a resposta para uma perginta, vc admite que não sabe.
                                                
Aqui está a pergunta: {input}""")

Passo 1: Criando os dicionários de prompts.

In [3]:
prompt_infos = [
    {
        'name':'Física',
        'description':'Ideal para responder perguntas sobre física',
        'prompt_template':fis_template
    },
    {
        'name':'Matemática',
        'description':'Ideal para responder perguntas sobre matemática',
        'prompt_template':mat_template
    },
    {
        'name':'História',
        'description':'Ideal para responder perguntas sobre história',
        'prompt_template':hist_template
    }
]

In [4]:
# instanciando o chat
chat = ChatOpenAI()

Passo 2: Criando as chains e armazenando-as num dicionário de chains.

In [7]:
# primeiro vamos criar um dicionário vazio
chains = {}

# usando um laço para criar as chains
# para cada dicionario dentro de prompt_infos
for info in prompt_infos:
    # crie uma chain utilizando seu respectivo prompt template
    chain = LLMChain(llm=chat, prompt=info['prompt_template'], verbose=True)
    # depois armazene essa chain no dicionário chains
    chains[info['name']] = chain

In [8]:
# vamos exibir nosso dicionario
chains

{'Física': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='\nVc é um exímio professor de física.\nVc é ótimo em responder perguntas sobre física de forma concisa e fácil de entender.Quando não sabe a resposta para uma perginta, vc admite que não sabe.\n                                                \nAqui está a pergunta: {input}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x70412a0164c0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x70412a01ec40>, openai_api_key=SecretStr('**********'), openai_proxy='')),
 'Matemática': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='\nVc é um exímio professor de matemática, a verdadeira reencarnação de Pitágoras.Vc é ótimo em responde

Passo 3: Criando os destinos. Aqui teremos simplesmente uma lista q traz o nome do prompt e sua finalidade. Essas informações vem do dicionário de prompts.

In [12]:
# imortamos um objeto q é um prompt já preparado para fazer o roteamento
# vamos exibi-lo
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [16]:
# veja no prompt acima q ele menciona para o modelo se orientar pelo destinos
# precisamos fornecer esses destinos para q o prompt funcione de forma eficiente
destinos = [f"{p['name']}: {p['description']}" for p in prompt_infos]

# precisamos transformar essa lista numa string única
# para ser consumida pelo prompt
destinos_str = "\n".join(destinos)
print(destinos_str)

Física: Ideal para responder perguntas sobre física
Matemática: Ideal para responder perguntas sobre matemática
História: Ideal para responder perguntas sobre história


**Passo 4: Criando o prompt de roteamento**

In [17]:
# criando o prompt fornecendo os destinos ao roteador importado
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinos_str
)

# veja agora como ficou nosso prompt final
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Física: Ideal para responder perguntas sobre física
Matemática: Ideal para responder perguntas sobre 

In [27]:
# agora precisamos criar um último prompt
# que é o q vai receber o input do usuario
router_template_final = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

# criando uma chain para esse prompt
router_chain = LLMRouterChain.from_llm(llm=chat, prompt=router_template_final, verbose=True)

In [23]:
# precisamos criar uma prompt/chain default
# ela será acionada quando o input do usuario não solicitar nenhuma das chains criadas
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=chat, prompt=default_prompt, verbose=True)

In [28]:
# e por ultimo, criando o encadeamento das chains
# q vai utilizar todos os elementos criados anteriormente nesta aula
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=chains,
    default_chain=default_chain,
    verbose=True)

In [26]:
# testando...
# veja o retorno do verbose, como o modelo 'pensa' e trabalha o roteamento
# cada pergunta é direcionada para a matéria específica
chain.invoke({'input':'O q são equações polinomiais? Me exmplique como se tivesse 5 anos'})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Matemática: {'input': 'O que são equações polinomiais? Me explique como se eu tivesse 5 anos.'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: 
Vc é um exímio professor de matemática, a verdadeira reencarnação de Pitágoras.Vc é ótimo em responder perguntas sobre matemática de forma concisa e fácil de entender.Quando não sabe a resposta para uma perginta, vc admite que não sabe.
                                                
Aqui está a pergunta: O que são equações polinomiais? Me explique como se eu tivesse 5 anos.

> Finished chain.

> Finished chain.


{'input': 'O que são equações polinomiais? Me explique como se eu tivesse 5 anos.',
 'text': 'Equações polinomiais são como um tipo de jogo de descobrir o número secreto. Você tem uma mistura de números e letras, como por exemplo 2x + 3 = 7. Para encontrar o valor de x, você precisa usar regras especiais, como somar e subtrair, até descobrir qual número faz a equação ficar certa. É como resolver um quebra-cabeça matemático!'}